# Imports

In [ ]:
from aoc import *
import re
import os
import itertools
import math


def Puzzle(day, year=2024):
    return AOCDPuzzle(year=year, day=day)


%load_ext line_profiler

# Day 1 - Historian Hysteria

In [ ]:
p = Puzzle(1)
p

In [ ]:
lines = p.input_data.splitlines()
pairs = [tuple(int(i) for i in line.split("  ")) for line in lines]
lists = [sorted([tpl[i] for tpl in pairs]) for i in range(2)]
p.answer_a = sum(abs(a - b) for a, b in zip(*lists))

In [ ]:
from collections import Counter

counts = Counter(lists[1])
p.answer_b = sum(val * counts[val] for val in lists[0])

# Day 2 - Red-Nosed Reports

In [ ]:
p = Puzzle(year=2024, day=2)
p

In [ ]:
reports = [vector(line) for line in p.input_data.splitlines()]


def is_safe(report):
    diffs = [(b - a) for a, b in itertools.pairwise(report)]
    return all(1 <= abs(d) <= 3 for d in diffs) and (
        all(d > 0 for d in diffs) or all(d < 0 for d in diffs)
    )


def signum(i):
    return -1 if i < 0 else 1 if i > 0 else 0


def dampen(report):
    return is_safe(report) or (
        any(is_safe(report[:i] + report[i + 1 :]) for i in range(len(report)))
    )


p.answer_a = len([r for r in reports if is_safe(r)])
p.answer_b = len([r for r in reports if dampen(r)])

# Day 3 - Null It Over

In [ ]:
p = Puzzle(day=3)
p

In [ ]:
p.answer_a = sum(
    int(x) * int(y) for x, y in re.findall(r"mul\((\d{1,3}),(\d{1,3})\)", p.input_data)
)

In [ ]:
def null_it_over_p2(input: str):
    instrux = re.findall(r"mul\((\d{1,3}),(\d{1,3})\)|(do|don't)\(\)", input)
    enabled = True
    sum = 0
    for x, y, mode in instrux:
        if mode == "do":
            enabled = True
        elif mode == "don't":
            enabled = False
        elif enabled:
            sum += int(x) * int(y)
    return sum


assert 48 == null_it_over_p2(
    """xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5)))"""
)

p.answer_b = null_it_over_p2(p.input_data)

# Day 4 - Ceres Search

In [ ]:
p = Puzzle(4)
p

In [ ]:
SEARCH_DIR = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]


def word_search(grid, p, dir, word) -> bool:
    for i in range(len(word)):
        q = (p[0] + dir[0] * i, p[1] + dir[1] * i)
        if grid.get(q) != word[i]:
            return False
    return True


def find_words(input: str, word="XMAS") -> int:
    grid = dict(
        ((y, x), c)
        for y, line in enumerate(input.splitlines())
        for x, c in enumerate(line)
    )
    xes = [p for p, c in grid.items() if c == "X"]
    answer = 0
    for x in xes:
        answer += sum(word_search(grid, x, dir, word) for dir in SEARCH_DIR)
    return answer


def is_x_mas(grid, a) -> bool:
    # Only diagonals!
    neighbors = [(-1, -1), (1, 1), (-1, 1), (1, -1)]
    letters = "".join(grid.get((a[0] + n[0], a[1] + n[1]), "") for n in neighbors)
    return len(letters) == 4 and letters in ("MSMS", "SMSM", "SMMS", "MSSM")


def find_x_mas(input: str) -> int:
    grid = dict(
        ((y, x), c)
        for y, line in enumerate(input.splitlines())
        for x, c in enumerate(line)
    )
    a_s = [p for p, c in grid.items() if c == "A"]
    return len([a for a in a_s if is_x_mas(grid, a)])


assert 18 == find_words(
    """MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX"""
)

p.answer_a = find_words(p.input_data)

assert 9 == find_x_mas(
    """MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX"""
)

p.answer_b = find_x_mas(p.input_data)

# Day 5 - Print Queue

In [ ]:
p = Puzzle(day=5)
p

In [ ]:
from collections import defaultdict
import functools


def sort_update(update, rules):
    def page_cmp(a, b):
        r1, r2 = rules[a], rules[b]
        if b in r1:
            return -1
        if a in r2:
            return 1
        return 0

    return tuple(sorted(update, key=functools.cmp_to_key(page_cmp)))


def update_is_sorted(update, rules):
    return update == sort_update(update, rules)


def parse_rules(input: str):
    rule_text, updates = input.split("\n\n")
    rules = defaultdict(set)
    for before, after in [
        vector(line.replace("|", ",")) for line in rule_text.splitlines()
    ]:
        rules[before].add(after)
    updates = [vector(line) for line in updates.splitlines()]
    return rules, updates


def print_jobs(input: str) -> int:
    rules, updates = parse_rules(input)
    total = 0
    for update in updates:
        is_sorted = update_is_sorted(update, rules)
        if is_sorted:
            mid = update[len(update) // 2]
            total += mid
    return total


def sort_jobs(input: str) -> int:
    rules, updates = parse_rules(input)
    total = 0
    for update in updates:
        new_rules = dict((k, v & set(update)) for k, v in rules.items() if k in update)
        supdate = sort_update(update, new_rules)
        if update != supdate:
            mid = supdate[len(supdate) // 2]
            total += mid

    return total


assert 143 == print_jobs(p.examples[0].input_data)

In [ ]:
p.answer_a = print_jobs(p.input_data)

In [ ]:
p.answer_b = sort_jobs(p.input_data)

# Day 6 - Guard Gallivant

In [ ]:
p = Puzzle(day=6)
p

In [ ]:
TURNS = {(-1, 0): (0, 1), (0, 1): (1, 0), (1, 0): (0, -1), (0, -1): (-1, 0)}


def parse_grid(input: str) -> dict:
    return dict(
        ((y, x), c)
        for y, line in enumerate(input.splitlines())
        for x, c in enumerate(line)
    )


def find_loop(grid: dict, pos: tuple, heading: tuple) -> bool:
    seen = set()
    save = pos
    try:
        grid[save] = "#"
        pos = (pos[0] - heading[0], pos[1] - heading[1])
        while True:
            if (pos, heading) in seen:
                return True
            seen.add((pos, heading))
            p = (pos[0] + heading[0], pos[1] + heading[1])
            c = grid.get(p)
            if c is None:
                return False
            if c == "#":
                heading = TURNS[heading]
                continue
            pos = p
        return False
    finally:
        grid[save] = "."


def guard_path(grid: dict) -> bool:
    guard = next(p for p, c in grid.items() if c == "^")
    heading = (-1, 0)
    path = set()
    seen = set()
    while True:
        if (guard, heading) in seen:
            raise ValueError("Loop detected")
        path.add(guard)
        seen.add((guard, heading))
        p = (guard[0] + heading[0], guard[1] + heading[1])
        c = grid.get(p)
        if c is None:
            break
        if c == "#":
            heading = TURNS[heading]
            continue
        guard = p
    return path


def guard_gallivant(input: str) -> int:
    return len(guard_path(parse_grid(input)))


def find_loops(input: str) -> int:
    grid = parse_grid(input)
    loops = 0
    pos = next(p for p, c in grid.items() if c == "^")
    heading = (-1, 0)
    seen = set()

    while True:
        seen.add(pos)
        p = (pos[0] + heading[0], pos[1] + heading[1])
        if p not in grid:
            break
        if grid.get(p) == "#":
            heading = TURNS[heading]
            p = (pos[0] + heading[0], pos[1] + heading[1])
        if p not in seen and find_loop(grid, p, heading):
            loops += 1
        pos = p

    return loops


assert (
    guard_gallivant(
        """....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#..."""
    )
    == 41
)

assert (
    find_loops(
        """....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#..."""
    )
    == 6
)

In [ ]:
p.answer_a = guard_gallivant(p.input_data)

In [ ]:
p.answer_b = find_loops(p.input_data)

# Day 7 - Bridge Repair

In [ ]:
p = Puzzle(day=7)
p

In [ ]:
import operator
import itertools

pow10_table = [10**i for i in range(1, 10)]


def concat(n1, n2):
    for k in pow10_table:
        if n2 - k < 0:
            return n1 * k + n2
    assert False, f"{n1} {n2}"


# First pass: brute force solution - ~15s on part 2
def solve_equation(answer: int, terms: list[int], operators) -> int:
    for ops in itertools.product(operators, repeat=len(terms) - 1):
        total = terms[0]
        for val, op in zip(terms[1:], ops):
            total = op(total, val)
            if total > answer:
                break
        else:
            if total == answer:
                return ops
    return None


# RTL recursive solver based on solutions seen on AoC Reddit.
def solver(answer: int, terms: list[int], use_concat=False) -> int:
    if len(terms) == 1:
        return terms[0] == answer
    if answer <= 0:
        return False
    head, tail = terms[:-1], terms[-1]
    return (
        # Addition
        (solver(answer - tail, head, use_concat) if answer > tail else False)
        # Multiplication
        or (solver(answer // tail, head, use_concat) if answer % tail == 0 else False)
        # Concatenation
        or (
            solver(answer // 10 ** len(str(tail)), head, use_concat)
            if (use_concat and str(answer).endswith(str(tail)))
            else False
        )
    )


def bridge_repair(input: str, use_concat=False) -> int:
    total = 0
    # operators = (operator.add, operator.mul) + (concat,) * use_concat
    for line in input.splitlines():
        answer, *terms = [int(i) for i in re.findall(r"\d+", line)]
        if solver(answer, terms, use_concat):
            total += answer
    return total


assert 3749 == bridge_repair(p.examples[0].input_data)

assert 11387 == bridge_repair(p.examples[0].input_data, True)

In [ ]:
p.answer_a = bridge_repair(p.input_data)

In [ ]:
p.answer_b = bridge_repair(p.input_data, True)

# Day 8: Resonant Collinearity

In [ ]:
p = Puzzle(day=8)
p

In [ ]:
def parse_antennas(input: str) -> dict:
    graph = parse_grid(input)
    antennas = defaultdict(list)
    for p, c in graph.items():
        if c.isdigit() or c.isalpha():
            antennas[c].append(p)
    return graph, antennas


def antinodes(input: str) -> int:
    graph, antennas = parse_antennas(input)
    antinodes = set()
    for nodes in antennas.values():
        for p, q in itertools.combinations(nodes, 2):
            dy, dx = q[0] - p[0], q[1] - p[1]
            for antinode in ((p[0] - dy, p[1] - dx), (q[0] + dy, q[1] + dx)):
                if antinode in graph:
                    antinodes.add(antinode)

    return len(antinodes)


def antinode_harmonics(input: str) -> int:
    graph, antennas = parse_antennas(input)
    height, width = max(graph.keys())
    antinodes = set()
    for nodes in antennas.values():
        for p, q in itertools.combinations(nodes, 2):
            dy, dx = q[0] - p[0], q[1] - p[1]
            for i in range(max(height, width) + 1):
                nodes = [
                    n
                    for n in (
                        (p[0] - dy * i, p[1] - dx * i),
                        (q[0] + dy * i, q[1] + dx * i),
                    )
                    if n in graph
                ]
                if not nodes:
                    break
                antinodes.update(nodes)

    return len(antinodes)


assert 14 == antinodes(p.examples[0].input_data)
assert 34 == antinode_harmonics(p.examples[0].input_data)

In [ ]:
p.answer_a = antinodes(p.input_data)

In [ ]:
p.answer_b = antinode_harmonics(p.input_data)

# Day 9 - Disk Fragmenter

In [ ]:
p = Puzzle(day=9)
p

In [ ]:
import heapq


def build_filesystem(input: str):
    used = []
    free = []
    offset = 0
    for i, c in enumerate(input):
        num_blocks = int(c)
        if not num_blocks:
            continue
        if i % 2 == 0:
            # Files are stored as (-end, -start, fileno)
            used.append((-offset - num_blocks, -offset, i // 2))
        else:
            # Free space represnted as (start, end, None)
            free.append((offset, offset + num_blocks, None))
        offset += num_blocks
    heapq.heapify(used)
    heapq.heapify(free)
    return used, free


def defrag(input: str) -> int:
    used, free = build_filesystem(input)
    while free and free[0][0] < -used[0][1]:  # min(free)[0] < -(min(used)[0]):
        end, start, fileno = heapq.heappop(used)
        fstart, fend, _ = heapq.heappop(free)
        file_size = start - end  # negative numbers
        free_blocks = fend - fstart
        moved_blocks = min(file_size, free_blocks)
        heapq.heappush(used, (-fstart - moved_blocks, -fstart, fileno))
        if moved_blocks < file_size:
            heapq.heappush(used, (end + moved_blocks, start, fileno))
        fstart += moved_blocks
        free_blocks -= moved_blocks
        if free_blocks > 0:
            heapq.heappush(free, (fstart, fend, None))
    total = 0
    for end, start, fileno in used:
        total += sum(fileno * n for n in range(-start, -end))
    return total


def defrag_wholefile(input: str) -> int:
    used, free = build_filesystem(input)
    free_blocks = sorted(free)
    result = dict()
    for end, start, fileno in sorted(used, key=lambda x: x[2], reverse=True):
        file_size = start - end
        free_block = None
        for i, h in enumerate(free_blocks):
            if h[0] > -start:
                break
            if h[1] - h[0] >= file_size:
                free_block = (i, h)
                break
        if free_block is None:  # No room
            result[fileno] = (-start, -end)
            continue
        i, (fstart, fend, _) = free_block
        assert fend - fstart >= file_size
        result[fileno] = (fstart, fstart + file_size)
        fstart += file_size
        if fstart < fend:
            free_blocks[i] = (fstart, fend, None)
        else:
            free_blocks.pop(i)
    total = 0
    for fileno, (start, end) in result.items():
        total += sum(fileno * n for n in range(start, end))
    return total


assert 1928 == defrag("2333133121414131402")
assert 2858 == defrag_wholefile("2333133121414131402")

In [ ]:
# p.answer_a =
print(p.answers)
defrag(p.input_data.strip())

In [ ]:
# p.answer_b =
defrag_wholefile(p.input_data.strip())

# Day 10 - Hoof It

In [ ]:
p = Puzzle(day=10)
p

In [ ]:
def moves(grid, p):
    start = int(grid[p])
    for q in neighbors4(p):
        if q in grid and grid[q].isdigit() and int(grid[q]) == start + 1:
            yield q


def score_and_rating(start, goals, moves):
    """Count number of distinct paths from start to any goal"""
    paths = 0
    reached = set()
    todo = [(start,)]
    while todo:
        path = todo.pop()
        p = path[-1]
        if p in goals:
            reached.add(p)
            paths += 1
            continue
        for q in moves(p):
            if q not in path:
                todo.append(path + (q,))
    return len(reached), paths


def hoof_it(input: str) -> int:
    grid = parse_grid(input)
    trailheads = [p for p, c in grid.items() if c == "0"]
    nines = [p for p, c in grid.items() if c == "9"]

    moves_func = lambda p: moves(grid, p)
    scores_and_ratings = [score_and_rating(t, nines, moves_func) for t in trailheads]
    scores = sum(s for s, r in scores_and_ratings)
    ratings = sum(r for s, r in scores_and_ratings)
    return scores, ratings


assert 36, 81 == hoof_it(
    """89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732"""
)

In [ ]:
p.answers = hoof_it(p.input_data)

# Day 11 - Plutonian Pebbles

In [ ]:
p = Puzzle(day=11)
p

In [ ]:
def pebble_rules(p):
    if p == 0:
        yield 1
    else:
        pstr = str(p)
        if len(pstr) % 2 == 0:
            yield int(pstr[: len(pstr) // 2])
            yield int(pstr[len(pstr) // 2 :])
        else:
            yield 2024 * p


def plutonian_pebbles(input: str, blinks: int) -> int:
    pebbles = vector(input)
    counts = Counter(pebbles)
    for _ in range(blinks):
        newcounts = Counter()
        for p, count in counts.items():
            for q in pebble_rules(p):
                newcounts[q] += count
        counts = newcounts
    return sum(counts.values())


assert 55312 == plutonian_pebbles("125 17", 25)

In [ ]:
p.answer_a = plutonian_pebbles(p.input_data, 25)

In [ ]:
p.answer_b = plutonian_pebbles(p.input_data, 75)

# Day 12 - Garden Groups

In [ ]:
p = Puzzle(day=12)
p

In [ ]:
def perimeter(graph, region) -> int:
    answer = 0
    for p in region:
        c = graph[p]
        answer += 4 - sum(1 for q in neighbors4(p) if graph.get(q) == c)
    return answer


def count_sides(region) -> int:
    sides = defaultdict(set)
    # Remove any completely surrounded points
    outside = set(p for p in region if not all(n in region for n in neighbors4(p)))
    # If a point has no neighbor above/below/left/right, add it to the appropriate side set
    for y, x in outside:
        if not (y - 1, x) in region:
            sides[("u", y)].add(x)
        if not (y + 1, x) in region:
            sides[("d", y)].add(x)
        if not (y, x - 1) in region:
            sides[("l", x)].add(y)
        if not (y, x + 1) in region:
            sides[("r", x)].add(y)
    num_sides = 0
    for _, points in sides.items():
        num_sides += sum(1 for p in points if p - 1 not in points)
    return num_sides


def find_regions(grid: dict) -> list:
    """Build a set of all contiguous regions"""
    seen = set()
    regions = []
    for p, c in grid.items():
        if p in seen:
            continue
        region = set([p])
        current = [p]
        while current:
            for n in neighbors4(current.pop()):
                x = grid.get(n)
                if x == c and n not in region:
                    region.add(n)
                    current.append(n)
        seen.update(region)
        regions.append(region)
    assert all(p in seen for r in regions for p in r)
    return regions


def garden_groups(input: str) -> int:
    grid = parse_grid(input)
    regions = find_regions(grid)
    total = sum(len(r) * perimeter(grid, r) for r in regions)
    bulk = sum(len(r) * count_sides(r) for r in regions)
    return total, bulk


assert 772, 436 == garden_groups(
    """OOOOO
OXOXO
OOOOO
OXOXO
OOOOO"""
)

assert 1930, 1206 == garden_groups(
    """RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE"""
)

assert 692, 236 == garden_groups(
    """EEEEE
EXXXX
EEEEE
EXXXX
EEEEE"""
)

In [ ]:
p.answers = garden_groups(p.input_data)

# Day 13 - Claw Contraption

In [ ]:
p = Puzzle(day=13)
p

In [ ]:
def claw_contraption(input: str, add=0) -> int:
    claws = input.split("\n\n")
    total = 0
    for claw in claws:
        ax, ay, bx, by, px, py = [int(x) for x in re.findall(r"\d+", claw)]
        px += add
        py += add
        # A million algebras later...
        divisor = ay * bx - ax * by
        m, mr = divmod(ay * px - ax * py, divisor)
        n, nr = divmod(bx * py - by * px, divisor)
        if mr != 0 or nr != 0:
            continue
        total += 3 * n + m
    return total


assert 480 == claw_contraption(p.examples[0].input_data)

In [ ]:
p.answer_a = claw_contraption(p.input_data)

In [ ]:
p.answer_b = claw_contraption(p.input_data, 10**13)

# Day 14 - Restroom Redoubt

In [ ]:
p = Puzzle(day=14)
p

In [ ]:
def parse_robots(input: str) -> list:
    for line in input.splitlines():
        x, y, dx, dy = [int(i) for i in re.findall(r"-?\d+", line)]
        yield ((y, x), (dy, dx))


def move_robot(p, d, width, height, rounds):
    q = ((p[0] + d[0] * rounds) % height, (p[1] + d[1] * rounds) % width)
    return q, d


def print_robots(robots, width, height):
    grid = Counter()
    for (y, x), _ in robots:
        grid[(y, x)] += 1
    for y in range(height):
        line = []
        for x in range(width):
            c = grid.get((y, x), 0)
            line.append("." if c == 0 else str(c % 10))
        print("".join(line))


def restroom_redoubt(
    input: str, width: int = 101, height: int = 103, rounds=100
) -> int:
    robots = parse_robots(input)
    robots = [move_robot(p, d, width, height, rounds) for p, d in robots]
    sectors = Counter()
    w2 = 1 + width // 2
    h2 = 1 + height // 2
    for (y, x), _ in robots:
        sx = divmod(x + 1, w2)
        sy = divmod(y + 1, h2)
        if sx == (1, 0) or sy == (1, 0):
            continue
        sector = 2 * sy[0] + sx[0]
        sectors[sector] += 1
    return multiply(sectors.values())


def robot_regions(robots: list) -> list:
    """Adapted logic from Day 12"""
    seen = set()
    grid = set(r for r, _ in robots)
    regions = []
    for p in grid:
        if p in seen:
            continue
        region = set([p])
        current = [p]
        while current:
            for n in neighbors4(current.pop()):
                if n in grid and n not in region:
                    region.add(n)
                    current.append(n)
        seen.update(region)
        regions.append(region)
    return regions


def find_easteregg(input: str, width: int = 101, height: int = 103):
    robots = list(parse_robots(input))
    i = 0
    while True:
        regions = robot_regions(robots)
        if any(len(r) > 100 for r in regions):
            print(f"Round {i}:")
            print_robots(robots, width, height)
            break
        robots = [move_robot(p, d, width, height, 1) for p, d in robots]
        i += 1
    return i


assert 12 == restroom_redoubt(p.examples[0].input_data, 11, 7)

In [ ]:
p.answer_a = restroom_redoubt(p.input_data)

In [ ]:
p.answer_b = find_easteregg(p.input_data)

# Day 15 - Warehouse Woes

In [ ]:
p = Puzzle(day=15)
p

In [ ]:
ROBOT_MOVES = {"^": (-1, 0), "v": (1, 0), "<": (0, -1), ">": (0, 1)}


def print_grid(grid):
    m = max(grid.keys())
    for y in range(m[0] + 1):
        for x in range(m[1] + 1):
            print(grid.get((y, x), "."), end="")
        print()


def warehouse_woes(input: str) -> int:
    grid, moves = input.split("\n\n")
    grid = parse_grid(grid)
    moves = [ROBOT_MOVES[m] for m in moves if m in ROBOT_MOVES]
    robot = [p for p, c in grid.items() if c == "@"][0]
    for m in moves:
        p = robot
        line = []
        while grid[p] in "@O.":
            line.append(p)
            if grid[p] == ".":
                break
            p = (p[0] + m[0], p[1] + m[1])
        if grid[line[-1]] != ".":
            continue
        for p, q in itertools.pairwise(reversed(line)):
            ".O"
            tmp = grid[p]
            grid[p] = grid[q]
            grid[q] = tmp
        assert grid[robot] == ".", f"{robot=} {line=}"
        robot = line[1]
    boxes = [p for p, c in grid.items() if c == "O"]
    return sum(100 * y + x for y, x in boxes)


assert 2028 == warehouse_woes(
    """########
#..O.O.#
##@.O..#
#...O..#
#.#.O..#
#...O..#
#......#
########

<^^>>>vv<v>>v<<"""
)

assert 10092 == warehouse_woes(
    """##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^"""
)

In [ ]:
p.answer_a = warehouse_woes(p.input_data)

In [ ]:
def move(grid, robot, m):
    # Left / right pushes and uncontended up/down are simple
    q = (robot[0] + m[0], robot[1] + m[1])
    if grid[q] == "#":
        return robot
    if m in ((0, -1), (0, 1)) or grid[q] == ".":
        line = []
        p = robot
        while grid[p] in "@[].":
            line.append(p)
            if grid[p] == ".":
                break
            p = (p[0] + m[0], p[1] + m[1])
        if grid[line[-1]] != ".":
            return robot
        for p, q in itertools.pairwise(reversed(line)):
            grid[p], grid[q] = grid[q], grid[p]
        assert grid[robot] == ".", f"{robot=} {line=}"
        return line[1]

    # A box is above or below
    assert m[1] == 0
    assert grid[q] in "[]"

    def next_level(grid, point, m):
        assert m[1] == 0
        p = point[0] + m[0], point[1] + m[1]
        c = grid[p]
        if c == "#":
            raise ValueError(f"Blocked at {p=}")
        if c == "]":
            yield (p[0], p[1] - 1)
            yield p
        elif c == "[":
            yield p
            yield (p[0], p[1] + 1)

    layers = [set([robot])]
    while True:
        try:
            next_layer = set(q for p in layers[-1] for q in next_level(grid, p, m))
            if not next_layer:
                break
            layers.append(next_layer)
        except ValueError:
            return robot

    nubot = robot[0] + m[0], robot[1] + m[1]
    for layer in reversed(layers):
        for p in layer:
            q = p[0] + m[0], p[1] + m[1]
            grid[q], grid[p] = grid[p], "."

    return nubot


def double_box(input: str) -> int:
    grid, moves = input.split("\n\n")
    grid = (
        grid.replace("#", "##").replace("O", "[]").replace(".", "..").replace("@", "@.")
    )
    grid = parse_grid(grid)

    moves = [ROBOT_MOVES[m] for m in moves if m in ROBOT_MOVES]
    robot = [p for p, c in grid.items() if c == "@"][0]
    for i, m in enumerate(moves):
        # print(f"{i=} {robot=} {m=}")
        robot = move(grid, robot, m)
    score = sum(100 * y + x for (y, x), c in grid.items() if c == "[")
    return score

In [ ]:
assert 9021 == double_box(
    """##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^"""
)

In [ ]:
p.answer_b = double_box(p.input_data)

# Day 16 - Reindeer Maze

In [ ]:
p = Puzzle(day=16)
p

In [ ]:
NORTH, SOUTH, EAST, WEST = (-1, 0), (1, 0), (0, 1), (0, -1)
MAZE_TURNS = {
    NORTH: [WEST, EAST],
    SOUTH: [EAST, WEST],
    WEST: [NORTH, SOUTH],
    EAST: [NORTH, SOUTH],
}


def reindeer_maze(input: str) -> int:
    maze = parse_grid(input)
    start = next(p for p, c in maze.items() if c == "S")
    finish = next(p for p, c in maze.items() if c == "E")

    def moves_func(tpl):
        p, d = tpl
        for move in [d] + MAZE_TURNS[d]:
            q = p[0] + move[0], p[1] + move[1]
            if maze.get(q) in "SE.":
                yield (q, move)

    def cost_func(t1, t2):
        p1, d1 = t1
        p2, d2 = t2
        return 1 * (p1 != p2) + 1000 * (d1 != d2)

    def path_cost(path):
        return sum(cost_func(t1, t2) for t1, t2 in itertools.pairwise(best_path))

    best_path = Astar(
        (start, EAST), moves_func, lambda tpl: 0 if tpl[0] == finish else 1, cost_func
    )
    assert best_path
    best_cost = path_cost(best_path)

    return best_cost


assert 7036 == reindeer_maze(p.examples[0].input_data)

In [ ]:
# p.answer_a = reindeer_maze(p.input_data)
reindeer_maze(
    """#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################"""
)

In [ ]:
p.answer_a = reindeer_maze(p.input_data)

In [ ]:
def make_move(p, d):
    return p[0] + d[0], p[1] + d[1]


def get_exits(maze, p):
    exits = set(d for d in (NORTH, SOUTH, EAST, WEST) if maze[make_move(p, d)] in ".")
    if (
        (len(exits) == 1 and maze[p] not in "SE")
        or exits == set([NORTH, SOUTH])
        or exits == set([EAST, WEST])
    ):
        return
    assert len(exits)
    return exits


def can_reach(maze, p, q):
    py, px = p
    qy, qx = q
    if py == qy:
        px, qx = min(px, qx), max(px, qx)
        return all(maze[(py, x)] in ".SE" for x in range(px, qx))
    if px == qx:
        py, qy = min(py, qy), max(py, qy)
        return all(maze[(y, px)] in ".SE" for y in range(py, qy))
    return False


def moves_func(maze, tpl):
    p, d = tpl
    for move in [d] + MAZE_TURNS[d]:
        q = p[0] + move[0], p[1] + move[1]
        if maze.get(q) in "SE.":
            yield (q, move)


def cost_func(t1, t2):
    p1, d1 = t1
    p2, d2 = t2
    return cityblock_distance(p1, p2) + 1000 * (d1 != d2)


def path_cost(path):
    return sum(cost_func(t1, t2) for t1, t2 in itertools.pairwise(path))


def reindeer_junction(input: str) -> int:
    maze = parse_grid(input)
    start = next(p for p, c in maze.items() if c == "S")
    finish = next(p for p, c in maze.items() if c == "E")

    junctions = dict((p, get_exits(maze, p)) for p, c in maze.items() if c == ".")
    empty = [p for p, e in junctions.items() if not e]
    for p in empty:
        junctions.pop(p)
    nodes = set(list(junctions.keys()) + [start, finish])
    print(f"Found {len(junctions)} junctions")
    # Find nearest neighbors / straight shots from every node
    rows = defaultdict(set)
    cols = defaultdict(set)
    for p in nodes:
        rows[p[0]].add(p)
        cols[p[1]].add(p)
    reachable = defaultdict(set)
    seen = set()
    for p in nodes:
        candidates = rows[p[0]] | cols[p[1]]
        for q in candidates:
            if p == q or (p, q) in seen:
                continue
            if can_reach(maze, p, q):
                reachable[p].add(q)
                reachable[q].add(p)
            seen.add((p, q))
            seen.add((q, p))

    print(f"{len(maze)=} {len(junctions)=} {reachable[start]} {reachable[finish]}")

    best_path = Astar(
        (start, EAST),
        lambda tpl: moves_func(maze, tpl),
        lambda tpl: 0 if tpl[0] == finish else 1,
        cost_func,
    )
    assert best_path
    best_cost = path_cost(best_path)

    print(best_cost, len(best_path))
    todo = deque()
    path = [(start, EAST, 0)]
    todo.append(path)
    seen = set()
    seats = set()
    while todo:
        path = todo.popleft()
        p, d, cost = path[-1]
        if p == finish:
            if cost == best_cost:
                path = [p for p, _, _ in path]
                for (py, px), (qy, qx) in itertools.pairwise(path):
                    if py == qy:
                        px, qx = min(px, qx), max(px, qx)
                        for x in range(px, qx + 1):
                            seats.add((py, x))
                    else:
                        py, qy = min(py, qy), max(py, qy)
                        for y in range(py, qy + 1):
                            seats.add((y, px))
            continue
        for n in reachable[p] - seen:
            if n[0] < p[0]:
                assert n[1] == p[1]
                heading = NORTH
            elif n[0] > p[0]:
                assert n[1] == p[1]
                heading = SOUTH
            elif n[1] < p[1]:
                assert n[0] == p[0]
                heading = WEST
            else:
                assert n[0] == p[0]
                heading = EAST
            new_cost = cost + cost_func((p, d), (n, heading))
            if new_cost > best_cost:
                continue
            todo.append(path + [(n, heading, new_cost)])
        seen.add(p)

    return len(seats)

In [ ]:
# p.answer_b =
reindeer_junction(p.input_data)

# Day 17 - Chronospatial Computer

IntCode, finally!

In [ ]:
p = Puzzle(day=17)
p

In [ ]:
class Computer:
    INSNS = ["ADV", "BXL", "BST", "JNZ", "BXC", "OUT", "BDV", "CDV"]

    def __init__(self, input):
        self.reg = [0, 0, 0]
        self.output = []
        self.ip = 0
        self.tape = []
        for line in input.splitlines():
            m = re.match(r"Register ([A-C]): (-?\d+)", line)
            if m:
                self.reg[ord(m.group(1)[0]) - ord("A")] = int(m.group(2))
            elif line.startswith("Program: "):
                self.tape = [int(x) for x in re.findall(r"\d+", line)]

    def run(self, A=None, B=None, C=None, trace=False):
        if A is not None:
            self.reg[0] = A
        if B is not None:
            self.reg[1] = B
        if C is not None:
            self.reg[2] = C
        self.ip = 0
        self.output = []
        while self.ip < len(self.tape):
            insn, opcode = self.tape[self.ip : self.ip + 2]
            if trace:
                opval = opcode
                if insn in (0, 6, 7):
                    opval = [opcode, 1 << self.combo(opcode)]
                elif insn in (2, 5):
                    opval = [opcode, self.combo(opcode) % 8]
                elif insn == 3:
                    opval = [self.reg[0], opcode]
                elif insn == 4:
                    opval = [self.reg[1], self.reg[2]]
                print(
                    f"{self.ip:2} {Computer.INSNS[insn]} {opval} {self.reg} {self.output}"
                )
            match insn:
                case 0:  # adv
                    self.reg[0] //= 1 << self.combo(opcode)
                case 1:  # bxl
                    self.reg[1] ^= opcode
                case 2:  # bst
                    self.reg[1] = self.combo(opcode) % 8
                case 3:  # jnz
                    if self.reg[0] != 0:
                        self.ip = opcode
                        continue
                case 4:  # bxc - ignores opcode
                    self.reg[1] ^= self.reg[2]
                case 5:  # out
                    val = self.combo(opcode) % 8
                    self.output.append(val)
                case 6:  # bdv
                    self.reg[1] = self.reg[0] // (1 << self.combo(opcode))
                case 7:  # cdv
                    self.reg[2] = self.reg[0] // (1 << self.combo(opcode))
                case _:
                    assert False, f"Invalid instruction {insn=} at {ip=}"

            self.ip += 2

        return self

    def combo(self, op: int):
        if 0 <= op <= 3:
            return op
        assert op < 7
        return self.reg[op - 4]

    def outstr(self) -> str:
        return ",".join(str(x) for x in self.output)


assert Computer("Register C: 9\nProgram: 2,6").run().reg[1] == 1
assert (
    Computer("Register B: 2024\nRegister C: 43690\nProgram: 4,0").run().reg[1] == 44354
)
assert (
    Computer("Register A: 2024\nProgram: 0,1,5,4,3,0").run().outstr()
    == "4,2,5,6,7,7,7,7,3,1,0"
)

assert (
    "4,6,3,5,6,3,5,2,1,0"
    == Computer(
        """Register A: 729
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0"""
    )
    .run()
    .outstr()
)

In [ ]:
c = Computer(p.input_data)
p.answer_a = c.run().outstr()

In [ ]:
# Sadly I didn't solve this one myself


def find_quine(c, A=0, depth=None):
    if depth is None:
        depth = len(c.tape) - 1
    assert depth >= 0
    for bit in range(8):
        a = A | (bit << 3 * depth)
        output = c.run(A=a, B=0, C=0).output
        if len(output) != len(c.tape):
            continue
        if output == c.tape:
            return a
        if output[depth] == c.tape[depth]:
            result = find_quine(c, a, depth - 1)
            if result is not None:
                return result
    return None


p.answer_b = find_quine(Computer(p.input_data))

# Day 18 - RAM Run

In [ ]:
p = Puzzle(day=18)
p

In [ ]:
def ram_run(input: str, size=70, t0=1024) -> int:
    bytes = [vector(line) for line in input.splitlines()]
    corrupted = dict((p, i) for i, p in enumerate(bytes) if i < t0)

    def moves_func(p):
        for dy, dx in ((-1, 0), (1, 0), (0, -1), (0, 1)):
            q = qy, qx = p[0] + dy, p[1] + dx
            if 0 <= qy <= size and 0 <= qx <= size and q not in corrupted:
                yield q

    start = (0, 0)
    finish = (size, size)
    path = Astar(start, moves_func, h_func=lambda p: 0 if p == finish else 1)
    return len(path) - 1


def find_blockage(input: str, size=70) -> tuple:
    bytes = [vector(line) for line in input.splitlines()]
    corrupted = dict((p, i) for i, p in enumerate(bytes))

    def moves_func(p, time):
        for dy, dx in ((-1, 0), (1, 0), (0, -1), (0, 1)):
            q = qy, qx = p[0] + dy, p[1] + dx
            if 0 <= qy <= size and 0 <= qx <= size and corrupted.get(q, 10**10) > time:
                yield q

    start = (0, 0)
    finish = (size, size)
    lo = 1024
    hi = len(bytes)
    while lo < hi:
        time = (hi + lo) // 2
        path = Astar(
            start,
            moves_func=lambda p: [q for q in moves_func(p, time)],
            h_func=lambda p: 0 if p == finish else 1,
        )
        if path is None:
            hi = time - 1
        else:
            lo = time

    answer = bytes[time]
    return ",".join(str(x) for x in answer)


assert 22 == ram_run(p.examples[0].input_data, 6, 12)

In [ ]:
p.answer_a = ram_run(p.input_data)

In [ ]:
p.answer_b = find_blockage(p.input_data)

# Day 19 - Linen Layouts

In [ ]:
p = Puzzle(day=19)
p

In [ ]:
@functools.cache
def count_ways(pattern: str, towels) -> bool:
    if pattern == "":
        return 1
    return sum(
        count_ways(pattern[len(t) :], towels) for t in towels if pattern.startswith(t)
    )


def linen_layouts(input: str) -> int:
    towels, patterns = input.split("\n\n")
    towels = tuple(towels.split(", "))
    patterns = patterns.splitlines()
    ways = [count_ways(p, towels) for p in patterns]
    return sum(1 for w in ways if w > 0), sum(ways)


assert (6, 16) == linen_layouts(p.examples[0].input_data)

In [ ]:
p.answers = linen_layouts(p.input_data)

# Day 20 - Race Condition

In [ ]:
p = Puzzle(day=20)
p

In [ ]:
def race_condition(input: str, cheat_duration=2, atleast=2, debug=False) -> int:
    grid = parse_grid(input)
    start = next(p for p, c in grid.items() if c == "S")
    finish = next(p for p, c in grid.items() if c == "S")
    nodes = [p for p, c in grid.items() if c in ".SE"]
    assert all(n in nodes for n in (start, finish))

    neighbors = defaultdict(set)
    for p in nodes:
        for n in neighbors4(p):
            if grid.get(n) in ".SE":
                neighbors[p].add(n)

    slow_path = Astar(
        start,
        moves_func=lambda p: neighbors[p],
        cost_func=cityblock_distance,
        h_func=lambda p: 0 if grid[p] == "E" else 1,
    )

    assert slow_path is not None

    height, width = [v + 1 for v in max(grid)]
    path_order = dict((p, i) for i, p in enumerate(slow_path))

    # Find the nodes within cheat_duration for every node on the path
    striking_distance = defaultdict(set)
    for i, p in enumerate(slow_path):
        py, px = p
        for dy in range(-cheat_duration, cheat_duration + 1):
            if not 0 <= py + dy < height:
                continue
            maxx = cheat_duration - abs(dy)
            for dx in range(-maxx, maxx + 1):
                if not 0 <= (px + dx) < width:
                    continue
                q = py + dy, px + dx
                if path_order.get(q, -1) > i:
                    striking_distance[p].add(q)

    cheat_saves = defaultdict(set)
    for i, p in enumerate(slow_path):
        for q in striking_distance[p]:
            j = path_order[q]
            assert j > i
            cheat_distance = cityblock_distance(p, q)
            assert cheat_distance <= cheat_duration
            saved = j - i - cheat_distance
            if saved < atleast:
                continue
            cheat_saves[saved].add((p, q))

    if debug:
        for k, v in sorted(cheat_saves.items()):
            print(f"{k}: {len(v)}")
    return cheat_saves


def count_saves(saves, atleast=0) -> int:
    return sum(len(v) for c, v in saves.items() if c >= atleast)


def day20():
    p = Puzzle(year=2024, day=20)
    assert 44 == count_saves(race_condition(p.examples[0].input_data))
    assert {
        50: 32,
        52: 31,
        54: 29,
        56: 39,
        58: 25,
        60: 23,
        62: 20,
        64: 19,
        66: 12,
        68: 14,
        70: 12,
        72: 22,
        74: 4,
        76: 3,
    } == dict(
        (saved, len(cheats))
        for saved, cheats in race_condition(
            p.examples[0].input_data, cheat_duration=20, atleast=50
        ).items()
    )

    p.answer_a = count_saves(race_condition(p.input_data, 2, 100))
    p.answer_b = count_saves(race_condition(p.input_data, 20, 100))
    print(p.answers)


def day20_thread():
    import sys
    import threading

    threading.stack_size(67108864)  # 64MB stack
    sys.setrecursionlimit(2**20)  # something real big
    # you actually hit the 64MB limit first
    # going by other answers, could just use 2**32-1

    # only new threads get the redefined stack size
    thread = threading.Thread(target=day20)
    thread.start()
    thread.join()


day20_thread()

# Day 21 - Keypad Conundrum

In [ ]:
p = Puzzle(day=21)
p

In [ ]:
NUMPAD = "789456123 0A"
DIRPAD = " ^A<v>"
DYDX = {"^": (-1, 0), "v": (1, 0), "<": (0, -1), ">": (0, 1)}


def calc_pairs(keypad):
    result = defaultdict(set)
    for k1, k2 in itertools.permutations(keypad, 2):
        if " " in (k1, k2):
            continue
        py, px = divmod(keypad.index(k1), 3)
        qy, qx = divmod(keypad.index(k2), 3)
        moves = "^" * (py - qy) + "v" * (qy - py) + "<" * (px - qx) + ">" * (qx - px)
        # Discard illegal permutations
        for steps in itertools.permutations(moves, len(moves)):
            y, x = py, px
            legal = []
            for move in steps:
                dy, dx = DYDX[move]
                y, x = y + dy, x + dx
                c = keypad[y * 3 + x]
                if c == " ":
                    break  # Can't do this
                legal.append(move)
            else:
                result[(k1, k2)].add("".join(legal))
    return result


NUMPAD_MOVES = calc_pairs(NUMPAD)
DIRPAD_MOVES = calc_pairs(DIRPAD)


def encode(code, depth=0, maxdepth=1):
    MOVES = NUMPAD_MOVES if depth == 0 else DIRPAD_MOVES
    answer = []
    for p1, p2 in itertools.pairwise("A" + code):
        print(f"{'  ' * depth}{p1, p2}: {MOVES[(p1, p2)]}")
        best = "" if p1 == p2 else None
        for segment in MOVES[(p1, p2)]:
            if depth < maxdepth:
                path = encode(segment + "A", depth + 1, maxdepth)
            else:
                path = segment
            if best is None or len(path) < len(best):
                best = path
            print(f"{'  ' * (1 + depth)}{path} | {best}")
        answer.append(best + "A")
    print(f"{'**' * depth}: {code} -> {answer}")
    return "".join(answer)


def keypad_conundrum(input: str) -> int:
    pass

In [ ]:
for k1, k2 in itertools.pairwise("A029A"):
    print(f"{k1} -> {k2}:")
    best = None
    for moves in NUMPAD_MOVES[(k1, k2)]:
        print(f"\t{moves}:")
        path = ""
        for b1, b2 in itertools.pairwise("A" + moves):
            for dmoves in DIRPAD_MOVES[(b1, b2)]:
                print(f"\t{b1, b2} {dmoves} {best}")
                path = path + dmoves + "A"
        if best is None or len(path) < len(best[1]):
            best = moves, path
        print(f"\t{k1, k2} {moves} {path} / {best}")

# Day 22 - Monkey Market

In [ ]:
p = Puzzle(day=22)
p

In [ ]:
def evolve(s: int) -> int:
    def mix(p, q):
        return p ^ q

    def prune(p):
        return p % 16777216

    s = prune(mix(s, s * 64))
    s = prune(mix(s, s // 32))
    s = prune(mix(s, s * 2048))
    return s


assert [
    123,
    15887950,
    16495136,
    527345,
    704524,
    1553684,
    12683156,
    11100544,
    12249484,
    7753432,
    5908254,
] == list(itertools.accumulate(range(10), initial=123, func=lambda p, q: evolve(p)))


def monkey_market(input: str, rounds=2000) -> int:
    buyers = [int(line) for line in input.splitlines()]
    part1 = 0
    signals = defaultdict(set)
    for b in buyers:
        chain = list(
            itertools.accumulate(range(rounds), initial=b, func=lambda s, _: evolve(s))
        )
        assert len(chain) == rounds + 1
        part1 += chain[-1]
        bananas = np.array(chain) % 10
        diff = np.diff(bananas)
        for n in range(3, len(diff)):
            bid = bananas[n + 1]
            if bid == 10:
                signal = tuple(diff[n - 3 : n + 1])
                signals[bid].add(signal)
        print(f"{b}: final={bananas[-1]} {len(signals)}")

    return part1


assert 37327623 == monkey_market(
    """1
10
100
2024"""
)

In [ ]:
p.answer_a = monkey_market(p.input_data)